In [13]:
#unet-train1.py
#Treina rede unet para segmentacao semantica de eliret
from PIL import Image
import pandas as pd
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import cv2; import numpy as np; np.random.seed(7); import sys
import tensorflow.keras as keras; from tensorflow.keras.models import *
from tensorflow.keras.layers import *; from tensorflow.keras.optimizers import *
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize, downscale_local_mean

In [55]:
tamanho_img=512

def leCsv(nomeDir,nomeArq):
  print("Lendo: ",nomeArq); arq=open(os.path.join(nomeDir,nomeArq),"r")
  lines=arq.readlines(); arq.close(); n=len(lines)
  nl,nc = 286,384
  AX=np.empty((n,nl,nc),dtype='uint8'); AY=np.empty((n,nl,nc),dtype='uint8')
  i=0
  for linha in lines:
    linha=linha.strip('\n'); linha=linha.split(';')
    nomeDir=''
    #print(os.path.join(nomeDir,linha[0]))
    AX[i]=Image.open(os.path.join(nomeDir,linha[0]))
    AX[i]= np.array(AX[i])
  #  print(AX[i])
  #  AX[i]=AX[i]*255/AX[i].max()
    AY_aux=str(pd.read_csv(os.path.join(nomeDir,linha[1])))

    AY_aux= AY_aux.split('\n')[1]
    AY_aux=AY_aux.split('\\t')
    AY_aux[0]=AY_aux[0].split(' ')[2]
    AY[i]=AX[i].copy()
    cv2.rectangle(AY[i], (int(AY_aux[0]),int(AY_aux[1])), (int(AY_aux[2]),int(AY_aux[3])), (255,255,255) ,2)

    AY[i]= np.array(AY[i])
 #   AY[ AY>=125 ] = 255
 #   AY[ AY<125] = 0
 #   AY[i]=AY[i]*255/AY[i].max()
 #   f = plt.figure()
 #   f.add_subplot(1,4,1); plt.imshow(AX[i],cmap="gray"); plt.axis('off')
 #   f.add_subplot(1,4,2); plt.imshow(AY[i],cmap="gray"); plt.axis('off')
 #   plt.show(block=True)
 #   ax1=np.float32(AX[i].reshape(nl,nc))/255.0
 #   ax2=np.float32(AY[i].reshape(nl,nc))/255.0
 #   f = plt.figure()
 #   f.add_subplot(1,4,1); plt.imshow(ax1,cmap="gray"); plt.axis('off')
 #   f.add_subplot(1,4,2); plt.imshow(ax2,cmap="gray"); plt.axis('off')
 #   plt.show(block=True)
    i=i+1

  ax= np.float32(AX)/255.0
  ay= np.float32(AY)/255.0 #Entre 0 e +1
 #   AY[i]=AY[i]*255/AY[i].max()

  return ax,ay


In [56]:
#<<<<<<<<<<<<<<<<<<<< main <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
bdDir = ""
ax, ay = leCsv(bdDir,"train.csv")
vx, vy = leCsv(bdDir,"test.csv")
#qx, qy = leCsv(bdDir,"teste.csv")
outDir = "."; os.chdir(outDir)

Lendo:  train.csv
Lendo:  test.csv


In [24]:
def impHistoria(history):
  print(history.history.keys())
  plt.plot(history.history['loss']); plt.plot(history.history['val_loss'])
  plt.title('model loss'); plt.ylabel('loss'); plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

In [57]:
def unet(input_size = (286,384,1)):
  n=64
  inputs = Input(input_size)
  conv2 = Conv2D(n, 3, activation = 'relu', padding = 'same' )(inputs)
  conv2 = Conv2D(n, 3, activation = 'relu', padding = 'same' )(conv2)
  conv9 = Conv2D(1, 1, activation = 'relu', padding = 'same' )(conv2)
  model = Model(inputs = inputs, outputs = conv9)
  model.compile(optimizer = Adam(learning_rate=1e-5), loss = 'mean_squared_error',metrics=['accuracy'])
  from tensorflow.keras.utils import plot_model
  plot_model(model, to_file='unet-train1.png', show_shapes=True)
  model.summary()
  return model


In [58]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

#Escolha entre comecar treino do zero ou continuar o treino de onde parou
model=unet()
#model = load_model("unet1.h5");


#history=model.fit(ax, ay, batch_size=1, epochs=100, verbose=2, validation_data=(vx,vy));



batch_size=1
reduce_lr = ReduceLROnPlateau(monitor='loss',
factor=0.9, patience=1, min_lr=0.0000000001, verbose=True)


#history=model.fit(datagen.flow(ax,ay,batch_size=batch_size),epochs=100, verbose=2, validation_data=(vx, vy),callbacks=[reduce_lr])


history=model.fit(ax, ay, batch_size=3, epochs=25, verbose=2, validation_data=(vx,vy));
model.save("/content/drive/MyDrive/unet5.h5");

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 286, 384, 1)]     0         
                                                                 
 conv2d_12 (Conv2D)          (None, 286, 384, 64)      640       
                                                                 
 conv2d_13 (Conv2D)          (None, 286, 384, 128)     73856     
                                                                 
 conv2d_14 (Conv2D)          (None, 286, 384, 1)       129       
                                                                 
Total params: 74625 (291.50 KB)
Trainable params: 74625 (291.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoc

KeyboardInterrupt: 

In [ ]:
#impHistoria(history); model.save("/content/drive/MyDrive/unet1.h5");
impHistoria(history);

In [ ]:
score = model.evaluate(ax, ay, verbose=0); print('Training loss:', score)
score = model.evaluate(vx, vy, verbose=0); print('Validation loss:', score)

In [ ]:
#unet-pred1.py
#Faz segmentacao de elipses e retangulos usando rede gerada pelo unet-train1.py
from PIL import Image
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import cv2; import numpy as np; np.random.seed(7)
import tensorflow.keras as keras
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout, Conv2D, Conv2DTranspose
from tensorflow.keras import optimizers
import sys; from sys import argv
from matplotlib import pyplot as plt

#<<<<<<<<<<<<<<<<<<< main <<<<<<<<<<<<<<<<<<<<<<
bdDir = ""
outDir = "."; os.chdir(outDir)
nome="zz"
#inImgX = nome+"x.png"; inImgY = nome+"y.png"
inImgX="/content/drive/MyDrive/Colab Notebooks/dataset_artigobase/HeLa/train/DIC-C2DH-HeLa/01/t010.tif"
inImgY="/content/drive/MyDrive/Colab Notebooks/dataset_artigobase/HeLa/train/DIC-C2DH-HeLa/01_ST/SEG/man_seg010.tif"
outImgG=nome+"g.png"; outImgB =nome+"saida.png"
arquivoRede = "/content/drive/MyDrive/unet1.h5"

model = load_model(os.path.join(outDir,arquivoRede))
QX=cv2.imread(os.path.join(bdDir,inImgX),0)
QY=cv2.imread(os.path.join(bdDir,inImgY),0)

#im.show()
#QY= np.array(QY)

print(QY)
nl=QX.shape[0]; nc=QX.shape[1]
#print(str(nl) +' '+ str(nc))
qx=np.float32(QX)/255.0 #Entre 0 e +1
qx2=qx
qx=qx.reshape(1, nl, nc, 1)

qp=model.predict(qx); qp=qp.reshape(nl,nc) # entre 0 e +1

QPG=255.0*qp; QPG=np.clip(QPG,0,255) # Entre 0 e 255
QPG=np.uint8(QPG); cv2.imwrite(os.path.join(outDir,outImgG),qp)

QPB=np.zeros((nl,nc),dtype='uint8'); QPB[ qp>=0.02] = 255
cv2.imwrite(os.path.join(outDir,outImgB),QPB)

f = plt.figure()
#f.add_subplot(1,4,1); plt.imshow(QX,cmap="gray"); plt.axis('off')
f.add_subplot(1,2,1); plt.imshow(QY*50000,cmap="gray"); plt.axis('off')
#f.add_subplot(1,4,3); plt.imshow(QPB,cmap="gray"); plt.axis('off')
f.add_subplot(1,2,2); plt.imshow(qp*255/.16720633,cmap="gray"); plt.axis('off')
plt.show(block=True)

print(QY.max())

In [ ]:
vx[12].mean()
vy[13].max()

img1=vx[7]
img2=vy[7]

f = plt.figure()
f.add_subplot(1,4,1); plt.imshow(img1,cmap="gray"); plt.axis('off')
f.add_subplot(1,4,2); plt.imshow(img2,cmap="gray"); plt.axis('off')

In [ ]:
255/.02966377

In [ ]:
def get_iou(ground_truth, pred):
    # coordinates of the area of intersection.
    ix1 = np.maximum(ground_truth[0], pred[0])
    iy1 = np.maximum(ground_truth[1], pred[1])
    ix2 = np.minimum(ground_truth[2], pred[2])
    iy2 = np.minimum(ground_truth[3], pred[3])

    # Intersection height and width.
    i_height = np.maximum(iy2 - iy1 + 1, np.array(0.))
    i_width = np.maximum(ix2 - ix1 + 1, np.array(0.))

    area_of_intersection = i_height * i_width

    # Ground Truth dimensions.
    gt_height = ground_truth[3] - ground_truth[1] + 1
    gt_width = ground_truth[2] - ground_truth[0] + 1

    # Prediction dimensions.
    pd_height = pred[3] - pred[1] + 1
    pd_width = pred[2] - pred[0] + 1

    area_of_union = gt_height * gt_width + pd_height * pd_width - area_of_intersection

    iou = area_of_intersection / area_of_union

    return iou

In [ ]:
  from tensorflow.keras.utils import plot_model

plot_model(model, to_file='unet-train1.png', show_shapes=True)


In [ ]:
acc=0
nl,nc=512,512
print(np.shape(qp))
for i in range(0,20):
  qx=vx[i].reshape(1, nl, nc, 1)
  qp=model.predict(qx); qp=qp.reshape(nl,nc) # entre 0 e +1

  acc += get_iou(vy[i],qp).mean()

In [ ]:
acc/21

In [ ]:
def get_iou(a, b, epsilon=1e-5):
    """ Given two boxes `a` and `b` defined as a list of four numbers:
            [x1,y1,x2,y2]
        where:
            x1,y1 represent the upper left corner
            x2,y2 represent the lower right corner
        It returns the Intersect of Union score for these two boxes.

    Args:
        a:          (list of 4 numbers) [x1,y1,x2,y2]
        b:          (list of 4 numbers) [x1,y1,x2,y2]
        epsilon:    (float) Small value to prevent division by zero

    Returns:
        (float) The Intersect of Union score.
    """
    # COORDINATES OF THE INTERSECTION BOX
    x1 = max(a[0], b[0])
    y1 = max(a[1], b[1])
    x2 = min(a[2], b[2])
    y2 = min(a[3], b[3])

    # AREA OF OVERLAP - Area where the boxes intersect
    width = (x2 - x1)
    height = (y2 - y1)
    # handle case where there is NO overlap
    if (width<0) or (height <0):
        return 0.0
    area_overlap = width * height

    # COMBINED AREA
    area_a = (a[2] - a[0]) * (a[3] - a[1])
    area_b = (b[2] - b[0]) * (b[3] - b[1])
    area_combined = area_a + area_b - area_overlap

    # RATIO OF AREA OF OVERLAP OVER COMBINED AREA
    iou = area_overlap / (area_combined+epsilon)
    return iou

In [ ]:
print(np.shape(qp))